<a href="https://colab.research.google.com/github/ronron-gh/ESP32_EdgeAI/blob/main/nnabla_convert_mnist_model_to_csrc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

nnablaをインストールする。

In [1]:
!pip install nnabla-ext-cuda114

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


numpyを1.23.0に下げて、pillowを9.1.0に上げる（そうしないと、この後のclassification.pyの実行でエラーになった。数カ月前まではnumpyの変更だけで動いていたので、今後もこのようなバージョンの不整合は発生するかもしれない）。

In [2]:
!pip install numpy==1.23.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install pillow==9.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 75.4 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


nnablaのサンプル群をダウンロードする。

In [3]:
!git clone https://github.com/sony/nnabla-examples.git

fatal: destination path 'nnabla-examples' already exists and is not an empty directory.


mnistサンプルプログラムのフォルダに移動し、学習を実行。

In [1]:
%cd nnabla-examples/image-classification/mnist-collection/

/content/nnabla-examples/image-classification/mnist-collection


In [2]:
!python classification.py -c cudnn -n lenet -o output

2023-02-03 23:50:40,784 [nnabla][INFO]: Initializing CPU extension...
2023-02-03 23:50:42,060 [nnabla][INFO]: Running in cudnn
2023-02-03 23:50:42,220 [nnabla][INFO]: Initializing CUDA extension...
2023-02-03 23:50:42,335 [nnabla][INFO]: Initializing cuDNN extension...
2023-02-03 23:50:49,631 [nnabla][INFO]: Saving output/lenet_result_epoch0.nnp as nnp
2023-02-03 23:50:49,631 [nnabla][INFO]: Saving <_io.StringIO object at 0x7fc0478e3dc0> as prototxt
2023-02-03 23:50:49,640 [nnabla][INFO]: Parameter save (.h5): <_io.BytesIO object at 0x7fc047895ef0>
2023-02-03 23:50:49,641 [nnabla][INFO]: Model file is saved as (.nnp): output/lenet_result_epoch0.nnp
2023-02-03 23:50:49,641 [nnabla][INFO]: DataSource with shuffle(True)
2023-02-03 23:50:49,642 [nnabla][INFO]: Getting label data from http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz.
train-labels-idx1-ubyte.gz: 100% 28.2k/28.2k [00:00<00:00, 38.4MB/s]
2023-02-03 23:50:49,841 [nnabla][INFO]: Getting label data done.
2023-02-03 23:

outputフォルダが作られていることを確認

In [3]:
!ls

args.py			 dcgan.py	README.md	  vae.py
_checkpoint_nnp_util.py  mnist_data.py	requirements.txt  vat.py
classification_bnn.py	 output		siamese.py
classification.py	 __pycache__	tmp.monitor


Cソースコードの出力先フォルダを作成。

In [4]:
!mkdir ./output_csrc

学習済みモデルファイル(.nnp)をCソースコードに変換。

In [5]:
!nnabla_cli convert -O CSRC -b 1 ./output/lenet_result.nnp ./output_csrc

2023-02-03 23:59:26,785 [nnabla][INFO]: Initializing CPU extension...
NNabla command line interface (Version:1.33.0, Build:230118094635)
2023-02-03 23:59:27,836 [nnabla][WARNING]: The export file format is 'CSRC' or 'SAVED_MODEL' that argument '--export-format' will have to be set!!!
Importing ./output/lenet_result.nnp
 Expanding Validation.
Using network [Validation].


変換の結果、次の４つのファイルが出来ていることを確認（その他のファイルは使わない）。  
Validation_inference.c  
Validation_inference.h  
Validation_parameters.c  
Validation_parameters.h

In [6]:
!ls output_csrc

GNUmakefile	      Validation_inference.c  Validation_parameters.c
Validation_example.c  Validation_inference.h  Validation_parameters.h


変換後のファイルをPCにダウンロードするために、Google Driveに一旦コピーする。

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r output_csrc /content/drive/MyDrive